In [1]:
import cv2
import mediapipe as mp
import numpy as np
import joblib
import pandas as pd
from scipy.ndimage import gaussian_filter1d
from mediapipe.tasks.python.vision.pose_landmarker import PoseLandmarker
from mediapipe.tasks import python as p
from mediapipe.tasks.python import vision


ModuleNotFoundError: No module named 'cv2'

In [3]:
modelo = joblib.load("modelo_clasificacion_actividades.pkl")

In [5]:
# Configuración del detector de MediaPipe
BaseOptions = mp.tasks.BaseOptions
PoseLandmarker = mp.tasks.vision.PoseLandmarker
PoseLandmarkerOptions = mp.tasks.vision.PoseLandmarkerOptions
VisionRunningMode = mp.tasks.vision.RunningMode

In [7]:
def normalizar_coordenadas(df):
    cadera_derecha = df[df['landmark_index'] == 23][['x', 'y', 'z']].mean()
    cadera_izquierda = df[df['landmark_index'] == 24][['x', 'y', 'z']].mean()
    cadera_centro = (cadera_derecha + cadera_izquierda) / 2
    
    df['x'] -= cadera_centro['x']
    df['y'] -= cadera_centro['y']
    df['z'] -= cadera_centro['z']
    
    hombro_derecho = df[df['landmark_index'] == 11][['x', 'y', 'z']].mean()
    hombro_izquierdo = df[df['landmark_index'] == 12][['x', 'y', 'z']].mean()
    torso_tamano = np.linalg.norm(hombro_derecho - hombro_izquierdo)
    
    df['x'] /= torso_tamano
    df['y'] /= torso_tamano
    df['z'] /= torso_tamano
    
    return df

In [9]:
def filtrar_datos(df):
    # Convertir las columnas x, y, z a tipo float
    df['x'] = df['x'].astype(float)
    df['y'] = df['y'].astype(float)
    df['z'] = df['z'].astype(float)
    
    # Aplicar el filtro gaussiano
    df['x'] = gaussian_filter1d(df['x'], sigma=2)
    df['y'] = gaussian_filter1d(df['y'], sigma=2)
    df['z'] = gaussian_filter1d(df['z'], sigma=2)
    
    return df


In [11]:
def calcular_angulo(p1, p2, p3):
    vector1 = p1 - p2
    vector2 = p3 - p2
    cos_theta = np.dot(vector1, vector2) / (np.linalg.norm(vector1) * np.linalg.norm(vector2))
    angulo = np.arccos(np.clip(cos_theta, -1.0, 1.0))
    return np.degrees(angulo)

In [13]:
def extract_landmarks_to_dataframe(detection_result):
    # Crear una lista para almacenar las coordenadas de cada landmark
    landmarks = []
    for pose_landmark in detection_result.pose_landmarks:
        for landmark in pose_landmark:
            landmarks.append([landmark.x, landmark.y, landmark.z, landmark.visibility])

    # Convertir la lista a un DataFrame de pandas
    df_landmarks = pd.DataFrame(landmarks, columns=['x', 'y', 'z', 'visibility'])
    
    # Agregar una columna de índice para los landmarks
    df_landmarks['landmark_index'] = df_landmarks.index
    return df_landmarks



In [15]:
def generar_caracteristicas(df):
    articulaciones_clave = [11, 12, 13, 14, 15, 16, 23, 24]
    
    velocidades = []
    angulos_codo_derecho = []
    angulos_codo_izquierdo = []
    angulos_tronco = []
    
    for articulacion in articulaciones_clave:
        actual = df[df['landmark_index'] == articulacion]
        if len(actual) > 1:
            vel_x = actual['x'].iloc[-1] - actual['x'].iloc[-2]
            vel_y = actual['y'].iloc[-1] - actual['y'].iloc[-2]
            vel_z = actual['z'].iloc[-1] - actual['z'].iloc[-2]
            velocidad = np.sqrt(vel_x**2 + vel_y**2 + vel_z**2)
            velocidades.append(velocidad)

    hombro_derecho = df[df['landmark_index'] == 11][['x', 'y', 'z']].values
    codo_derecho = df[df['landmark_index'] == 13][['x', 'y', 'z']].values
    muneca_derecha = df[df['landmark_index'] == 15][['x', 'y', 'z']].values
    
    if hombro_derecho.size > 0 and codo_derecho.size > 0 and muneca_derecha.size > 0:
        angulos_codo_derecho.append(calcular_angulo(hombro_derecho[0], codo_derecho[0], muneca_derecha[0]))
    
    hombro_izquierdo = df[df['landmark_index'] == 12][['x', 'y', 'z']].values
    codo_izquierdo = df[df['landmark_index'] == 14][['x', 'y', 'z']].values
    muneca_izquierda = df[df['landmark_index'] == 16][['x', 'y', 'z']].values
    
    if hombro_izquierdo.size > 0 and codo_izquierdo.size > 0 and muneca_izquierda.size > 0:
        angulos_codo_izquierdo.append(calcular_angulo(hombro_izquierdo[0], codo_izquierdo[0], muneca_izquierda[0]))
    
    if hombro_derecho.size > 0 and hombro_izquierdo.size > 0:
        cadera_centro = (df[df['landmark_index'] == 23][['x', 'y', 'z']].values +
                         df[df['landmark_index'] == 24][['x', 'y', 'z']].values) / 2
        angulos_tronco.append(calcular_angulo(hombro_derecho[0], cadera_centro[0], hombro_izquierdo[0]))

    return [
        np.mean(velocidades), np.std(velocidades),
        np.mean(angulos_codo_derecho), np.std(angulos_codo_derecho),
        np.mean(angulos_codo_izquierdo), np.std(angulos_codo_izquierdo),
        np.mean(angulos_tronco), np.std(angulos_tronco)
    ]

In [ ]:
# Inicializar el detector
base_options = p.BaseOptions(model_asset_path='pose_landmarker_heavy.task')
options = vision.PoseLandmarkerOptions(
    base_options=base_options,
    output_segmentation_masks=True)
print(base_options)
detector = vision.PoseLandmarker.create_from_options(options)

cap = cv2.VideoCapture(0)
while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    # Procesamiento de la imagen para obtener landmarks
    rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    mp_image = mp.Image(image_format=mp.ImageFormat.SRGB, data=rgb_frame)
    detection_result = detector.detect(mp_image)

    df_landmarks = extract_landmarks_to_dataframe(detection_result)  # Asegúrate de que se incluyan todas las columnas requeridas
    df_landmarks = normalizar_coordenadas(df_landmarks)
    df_landmarks = filtrar_datos(df_landmarks)
    
    caracteristicas = generar_caracteristicas(df_landmarks)
    prediccion = modelo.predict([caracteristicas])[0]

    # Mostrar la predicción
    cv2.putText(frame, f'Prediccion: {prediccion}', (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
    cv2.imshow('Prediccion en tiempo real', frame)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()

BaseOptions(model_asset_path='pose_landmarker_heavy.task', model_asset_buffer=None, delegate=None)


C:\Users\mateo\AppData\Roaming\Python\Python312\site-packages\numpy\core\fromnumeric.py:3504: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
C:\Users\mateo\AppData\Roaming\Python\Python312\site-packages\numpy\core\_methods.py:129: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)
C:\Users\mateo\AppData\Roaming\Python\Python312\site-packages\numpy\core\_methods.py:206: RuntimeWarning: Degrees of freedom <= 0 for slice
  ret = _var(a, axis=axis, dtype=dtype, out=out, ddof=ddof,
C:\Users\mateo\AppData\Roaming\Python\Python312\site-packages\numpy\core\_methods.py:163: RuntimeWarning: invalid value encountered in divide
  arrmean = um.true_divide(arrmean, div, out=arrmean,
C:\Users\mateo\AppData\Roaming\Python\Python312\site-packages\numpy\core\_methods.py:198: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)
C:\Users\mateo\AppData\Roaming\Python\Python312\si